# IVCSD Housing Support Form Data Analysis

Welcome to the IVCSD Housing Support Form Data Analysis notebook. In this notebook, we will be automatically analyzing the data provided by IVCSD. All graphs generated will be saved as a picture to the `output` folder. Please run the `setup` code cell first. Then for each analysis you want, you can simply run the corresponding code block under each heading. If you do not see the custom theme, run it again to refresh inline styles.

## Get the Data

### Support Form Data

Please contact your supervisor for access to the form. You can insert it as a `.csv` into the `data` folder.

## Setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from aquarel import load_theme

from wordcloud import WordCloud
from nltk.corpus import stopwords

YEAR=2025

DEFINE_THEME = "arctic_light"
theme = load_theme(DEFINE_THEME)

nltk.download('stopwords')

dataForm = pd.read_csv("data/tracker.csv")
dataForm = dataForm.set_index("Assigned Case #")
dataForm["Inquiry Submitted On"] = pd.to_datetime(dataForm["Inquiry Submitted On"], format="%m/%d/%Y")
dataForm = dataForm[dataForm["Inquiry Submitted On"].dt.year == YEAR]

# %matplotlib inline

## Monthly Requests

In [ ]:
%matplotlib inline

fig, ax = plt.subplots(figsize=(22, 8))

_x = pd.date_range(start=pd.to_datetime(f"{YEAR}-01-01"), end=pd.to_datetime(f"{YEAR}-12-31"), freq="ME")
_y = []

for m in _x:
    dataInMonth = dataForm[dataForm["Inquiry Submitted On"].dt.month == m.month]
    _y.append(dataInMonth["Inquiry Submitted On"].count())

labels = _x.strftime("%b")

ax.plot(labels, _y)
ax.set_title(f"Number of Requests by Month for {YEAR}")
ax.set_ylabel("Number of Requests")
ax.set_xlabel("Month")

theme.apply()

fig.savefig("output/monthly_requests.png", facecolor=fig.get_facecolor())

## Requestor Type

In [ ]:
%matplotlib inline

fig, ax = plt.subplots(figsize=(12, 10))

roles = dataForm["Your Role"].value_counts()

roles.rename("Type of Requestors", inplace=True)
roles.plot(kind="pie", ax=ax, autopct='%1.1f%%', startangle=90, legend=False)

theme.apply()

fig.savefig("output/requestor_types.png", facecolor=fig.get_facecolor())

## Prefered Language

In [ ]:
%matplotlib inline

fig, ax = plt.subplots(figsize=(12, 10))

language = dataForm["Preferred language of communication"].value_counts()

language.rename("Preferred Language", inplace=True)
language.plot(kind="pie", ax=ax, autopct='%1.1f%%', startangle=90, legend=False)

theme.apply()

fig.savefig("output/language_preference.png", facecolor=fig.get_facecolor())

## Source of Information

In [ ]:
%matplotlib inline

fig, ax = plt.subplots(figsize=(12, 10))

source = dataForm["How Did You Hear About Our Mediation Services? "]
source = source.str.lower()
source = source.str.replace(" ", "")
source = source.str.split("\n")
source = source.explode()

source = source.value_counts()

source.rename("How did you Hear about Us?", inplace=True)
source.plot(kind="pie", ax=ax, autopct='%1.1f%%', startangle=90, legend=False)

theme.apply()

fig.savefig("output/source.png", facecolor=fig.get_facecolor())

## Requested Support

In [ ]:
%matplotlib inline

fig, ax = plt.subplots(figsize=(12, 10))

requestedSupport = dataForm["How can we best support you?"]

requestedSupport = requestedSupport.str.lower()
requestedSupport = requestedSupport.str.split("\n")
requestedSupport = requestedSupport.explode()

requestedSupport = requestedSupport.value_counts()

requestedSupport.rename("What kind of Support do you Need?", inplace=True)
requestedSupport.plot(kind="barh", ax=ax)

theme.apply()

fig.savefig("output/requested_support.png", facecolor=fig.get_facecolor())

## Category of Issue

In [ ]:
%matplotlib inline

fig, ax = plt.subplots(figsize=(12, 10))

issue = dataForm["Nature of Concern"]


issue = issue.str.lower()
issue = issue.str.split("\n")
issue = issue.explode()

otherText = ""
for i in issue:
    if ":" not in str(i):
        issue.loc[issue == i] = "other"
        otherText += str(i) + " "

issue = issue.value_counts()

issue.rename("What kind of Support do you Need?", inplace=True)
issue.plot(kind="pie", ax=ax, autopct='%1.1f%%', startangle=90, legend=False)

theme.apply()

otherCloud = WordCloud(stopwords=stopwords.words('english')).generate(otherText)
otherCloud.to_file("output/issue_category_wordcloud.png")

fig.savefig("output/issue_category.png", facecolor=fig.get_facecolor())




## Concerns Word Cloud

In [ ]:
fig, ax = plt.subplots(figsize=(12, 10))

concern = dataForm[["Other (Please specify):", "Please describe your concern"]]

concern = concern.apply(lambda x: " ".join(x.dropna().astype(str)),
                        axis=1)

text = ""
for i in concern:
    text += str(i) + " "

concernCloud = WordCloud(stopwords=stopwords.words('english')).generate(text)
concernCloud.to_file("output/concern_wordcloud.png")

## Urgency

In [ ]:
%matplotlib inline

fig, ax = plt.subplots(figsize=(12, 10))

urgency = dataForm["How urgent is your situation"].dropna()
urgency = urgency.str.split(":")
urgency = urgency.to_list()

urgency = [i[0] for i in urgency]
urgency = pd.Series(urgency)

urgency = urgency.value_counts()
urgency.plot(kind="bar", ax=ax)

ax.set_title("How Urgent is your Situtation?")
ax.set_xlabel("Urgency")
ax.set_ylabel("Count")

theme.apply()

fig.savefig("output/urgency.png", facecolor=fig.get_facecolor())

## Communication with Other Party (and their Role)

In [ ]:
%matplotlib inline

fig, ax = plt.subplots(figsize=(12, 10))

otherCommunication = dataForm["Have you already communicated with the other party?"].dropna()

otherCommunication = otherCommunication.str.lower()
otherCommunication = otherCommunication.str.replace("\"", "")
otherCommunication = otherCommunication.str.split(", ")
otherCommunication = otherCommunication.explode()

otherCommunication = otherCommunication.value_counts()

otherCommunication.drop("yes", inplace=True)

otherCommunication.rename("", inplace=True)
otherCommunication.plot(kind="pie", ax=ax, autopct='%1.1f%%', startangle=90, legend=False)

ax.set_title("Did you communicate with the other party?")

theme.apply()

fig.savefig("output/other_party_communication.png", facecolor=fig.get_facecolor())

fig, ax = plt.subplots(figsize=(12, 10))

otherPartyRole = dataForm["Their Role"].dropna()
otherPartyRole = otherPartyRole.value_counts()

otherPartyRole.rename(index={"Property Owner/Manager/Landlord": "Landlord et. al."}, inplace=True)
otherPartyRole.rename("", inplace=True)
otherPartyRole.plot(kind="bar", ax=ax)

ax.set_title("Other Party's Role")

theme.apply()

fig.savefig("output/other_party_role.png", facecolor=fig.get_facecolor())



## Response from Inquirer

In [ ]:
%matplotlib inline

fig, ax = plt.subplots(figsize=(12, 10))

response = dataForm["Response from inquirer \n"].dropna()

response = response.str.lower()
response = response.str[0]

response = response.value_counts() 
response.rename("Did the requestor respond?", inplace=True)
response.plot(kind="pie", ax=ax, autopct='%1.1f%%', startangle=90, legend=False)

theme.apply()

fig.savefig("output/requestor_responded.png", facecolor=fig.get_facecolor())

## Mediation Occurrence

In [ ]:
%matplotlib inline

fig, ax = plt.subplots(figsize=(12, 10))

mediation = dataForm["Mediation Occurrence (Y/N)"].dropna()

mediation = mediation.str.lower()

mediation = mediation.value_counts() 
mediation.rename("Did mediation happen?", inplace=True)
mediation.plot(kind="pie", ax=ax, autopct='%1.1f%%', startangle=90, legend=False)

theme.apply()

fig.savefig("output/mediation_happened.png", facecolor=fig.get_facecolor())

## Number of Followups

In [ ]:
%matplotlib inline

fig, ax = plt.subplots(figsize=(12, 10))

followups = dataForm["# of Follow Ups Completed"].dropna().astype(int)

followups = followups.apply(lambda x: "4+" if x >= 4 else x)

followups = followups.value_counts()

followups = pd.DataFrame({"label": followups.index, "count": followups.values})
followups.set_index("label", inplace=True)
followups = followups.sort_index()

followups.plot(kind="bar", ax=ax, legend=False)
ax.set_title("How many followups per case?")

theme.apply()

fig.savefig("output/followups.png", facecolor=fig.get_facecolor())

## Legal Aid Provided

In [ ]:
%matplotlib inline

fig, ax = plt.subplots(figsize=(12, 10))

legalAid = dataForm["Legal Advice Provided\n"].dropna()

legalAid = legalAid.str.lower()

for i in legalAid:
    if str(i)[0] != "n":
        legalAid.loc[legalAid == i] = "yes"
    else:
        legalAid.loc[legalAid == i] = "no"

legalAid = legalAid.value_counts() 
legalAid
legalAid.rename("Was legal aid provided?", inplace=True)
legalAid.plot(kind="pie", ax=ax, autopct='%1.1f%%', startangle=90, legend=False)

theme.apply()

fig.savefig("output/legaladvice.png", facecolor=fig.get_facecolor())

In [ ]:
fig, ax = plt.subplots(figsize=(12, 10))

legalType = dataForm[["Legal Issue Type\n"]]

legalType = legalType.apply(lambda x: " ".join(x.dropna().astype(str)),
                        axis=1)

legalType

text = ""
for i in legalType:
    text += str(i) + " "

legalType = WordCloud(stopwords=stopwords.words('english')).generate(text)
legalType.to_file("output/legal_type_wordcloud.png")

## Issues Bought to Mediation

In [ ]:
%matplotlib inline

fig, ax = plt.subplots(figsize=(12, 10))

iToM = dataForm[["Nature of Concern", "Mediation Occurrence (Y/N)"]].dropna()

iToM = iToM[iToM["Mediation Occurrence (Y/N)"].str.lower() == "y"]

iToM = iToM.value_counts()

iToM.rename("Issues to Mediation", inplace=True)
iToM.plot(kind="bar", ax=ax)

theme.apply()

fig.savefig("output/issues_to_mediation.png", facecolor=fig.get_facecolor())